In [1]:
import pandas as pd
import time
import numpy as np
import copy
import glob
import sys, os
import torch
import torch.nn as nn
import torch.utils.data as data_utils
from sklearn.metrics import precision_score, recall_score
import pickle
from sklearn import preprocessing
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from folktables import ACSDataSource, ACSEmployment,ACSIncome
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
from sklearn.metrics import roc_curve
from sklearn.model_selection import StratifiedKFold, train_test_split
from torchvision import models
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
import os
from torch.utils.data import Dataset
import numpy as np
import torch.nn.functional as F
import pandas as pd
from scipy.stats import multivariate_normal
import torch, random, copy, os
from collections import OrderedDict


In [2]:
# !pip install ray

# data preparation

In [3]:
import torch

if torch.cuda.is_available():
    print("CUDA is available.")
else:
    # CUDA is not available
    print("CUDA is not available. Running on CPU.")


CUDA is not available. Running on CPU.


In [4]:
os.getcwd()

'/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB'

In [5]:
 
class DeepNet(nn.Module):
    def __init__(self):
        super().__init__()
        # 14 : input shape
        # 9-> we have 9 columns in data 
        self.layer1 = nn.Linear(9, 512)
        self.act1 = nn.ReLU()
        self.dropout1 = nn.Dropout(p=0.5)
        self.layer2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        self.layer3 = nn.Linear(256, 60)
        self.act3 = nn.ReLU()
        self.output = nn.Linear(60, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.act1(self.layer1(x))
        x = self.dropout1(x)
        x = self.act2(self.layer2(x))
        x = self.act3(self.layer3(x))
        x = self.sigmoid(self.output(x))
        return x



# assigning Models

In [6]:
#create global model and client models
temp_model = DeepNet()
path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model.pt"
torch.save(temp_model.state_dict(), path)

# this has to be same in follwing code as well
selected_clients=[0,1]

for client_id in selected_clients:
    client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{client_id}_model.pth"
    torch.save(temp_model.state_dict(), client_model_path)
    
    


In [12]:

class Client:
    def __init__(self):
        self.client_id: int = None
        self.valset: DataLoader = None
        self.trainset: DataLoader = None
        self.testset: DataLoader = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
        print(self.device)
        self.model = DeepNet().to(self.device)
        self.criterion = torch.nn.BCELoss()

    def get_client_local_dataset(self):
        
        print("self.client_id:: ",self.client_id)

        temp_path_data="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/client_df"
        
        with open(temp_path_data+"/clients_training.pkl", "rb") as f:
            self.trainset = pickle.load(f)
            
        with open(temp_path_data+"/clients_validation.pkl", "rb") as f:
            self.valset = pickle.load(f)
         
        with open(temp_path_data+"/clients_testing.pkl", "rb") as f:
            self.testset  = pickle.load(f)

        self.trainset = self.trainset[self.client_id]
        self.valset = self.valset[self.client_id]       
        self.testset = self.testset[self.client_id]

    def train(self, client_id: int, model_params: OrderedDict[str, torch.Tensor], num_epochs=5, learning_rate=0.001):
        self.client_id = client_id
        self.get_client_local_dataset()

        # Define loss function and optimizer
        criterion = nn.BCELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

        for epoch in range(num_epochs):
            # Set the model to training mode
            self.model.train()
            print("epoch",epoch)

            for inputs, labels in self.trainset:
                inputs, labels = inputs.to(self.device), labels.to(self.device)
                optimizer.zero_grad()
                outputs = self.model(inputs)
                loss = self.criterion(outputs, labels.float())
                loss.backward()
                optimizer.step()

            # Calculate and print the training accuracy for this epoch (optional)
            correct = 0
            total = 0
            size = 0
            loss = 0
            correct = 0
            total = 0
            predicted_labels = []
            true_labels = []
            
            with torch.no_grad():
                for inputs, labels in self.valset:
                    inputs, labels = inputs.to(self.device), labels.to(self.device)
                    
                    outputs = self.model(inputs)
                    loss += self.criterion(outputs, labels)
                    predicted = outputs > 0.5

                    predicted_labels.extend(predicted.cpu().numpy())
                    true_labels.extend(labels.cpu().numpy())

                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()

            loss = loss / len(self.valset)        
            accuracy = correct / total
            
            # accuracy = 100 * correct / total
            print(f"Epoch {epoch+1}/{num_epochs}, Training Accuracy: {accuracy:.2f}%")

        # Optionally, save the trained model parameters
        client_model_path = f"/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/client_{self.client_id}_model.pth"
        torch.save(self.model.state_dict(), client_model_path)

        # Return the trained model parameters
        return list(self.model.state_dict().values()), len(self.trainset.dataset),self.model.state_dict()

    @torch.no_grad()
    def client_evaluate(self, val=False):
        
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []

        selected_clients=[0,1]
        models_directory = "/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"

        #dataset
        if val==True:
            dataset=self.valset
            print("------validation starts--------")
        else:
            dataset=self.testset
            print("--------Testing starts-------")
                    
        for client_id in selected_clients:
            
            print("=============================================")
            print(" selected client for Testing:: ", client_id)
            model_path = os.path.join(models_directory, f"client_{client_id}_model.pth")            
        
            self.model.load_state_dict(torch.load(model_path,map_location=self.device))

            self.model.eval()

            for inputs, targets in dataset:
                
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                
                outputs = self.model(inputs)
                loss += self.criterion(outputs, targets)
                predicted = outputs > 0.5
                
                predicted_labels.extend(predicted.cpu().numpy())
                true_labels.extend(targets.cpu().numpy())
                
                total += targets.size(0)
                correct += (predicted == targets).sum().item()
                
            
            loss = loss / len(self.testset)
            acc = correct / total
            
            # print("loss: %f\n" % (loss))
            
            print(f" Accuracy: {acc:.2%}")
    #         print(predicted_labels," :: ",true_labels)
            precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
            
            recall = recall_score(true_labels, predicted_labels)
    
            print(f"Precision: {precision:.2%}")
            print(f"Recall: {recall:.2%}")
            
            print("\n\n")
            
        return loss, acc

    
    @torch.no_grad()
    def server_evaluate(self):

        print("Global Model testing Starts")
        print("kindly check the Path. Select it based on FedAvg Model")
        
        path="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model.pt"
        path_2="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/global_model_2.pt"
        
        self.model.load_state_dict(torch.load(path_2,map_location=self.device))
        self.model.eval()
        size = 0
        loss = 0
        correct = 0
        total = 0
        predicted_labels = []
        true_labels = []
        
        for inputs, targets in self.testset:
            
            inputs, targets = inputs.to(self.device), targets.to(self.device)
            
            outputs = self.model(inputs)
            loss += self.criterion(outputs, targets)
            predicted = outputs > 0.5
            
#             print("predicted",predicted)
            predicted_labels.extend(predicted.cpu().numpy())
            true_labels.extend(targets.cpu().numpy())
            
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
            
        
        loss = loss / len(self.testset)
        acc = correct / total
        
        # print("loss: %f\n" % (loss))
        
        print(f"Global Testing Accuracy: {acc:.2%}")
        precision = precision_score(true_labels, predicted_labels,zero_division=0.0)
        
        recall = recall_score(true_labels, predicted_labels)
        
        print(f"Global Precision: {precision:.2%}")
        print(f"Global Recall: {recall:.2%}")
        
        return loss, acc
                                            

In [16]:
class Serverbase:
    def __init__ (self,model):
        self.model = model
        self.global_model=model
        self.num_rounds=5
        self.local_epoch=10
        self.optimizer=2
        self.lr=0.001
        self.beta=0.2
        self.batch_size=32
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

        self.client_id_indices, self.client_num_in_total = [0,1], 2 
        
        self.updated_params_cache = []
        self.weights_cache = []
        self.model_dict = []
        
        self.global_params_dict: OrderedDict[str : torch.Tensor] = None

        self.backbone=DeepNet
        _dummy_model = self.backbone()
        self.global_params_dict = OrderedDict(_dummy_model.state_dict())

        self.temp_dir="/home/chiragapandav/Downloads/Hiwi/Improving-Fairness-via-Federated-Learning/FedFB/models/"

        self.fair_global_t_step=[]
        self.fair_local_client=[]   
        self.acc_clients=[]
        
        
    def fairFed(self, num_rounds = 2, local_epochs = 2, learning_rate = 0.005, beta = 0.3, alpha = 0.1, optimizer = 'adam'):
        
        for round_ in tqdm(range(num_rounds)):
  
            selected_clients=[0,1]
            
            for client_id in selected_clients:
                client = Client()

                # weight is length of dataset
                updated_params_list, weight, model_dict_list = client.train(client_id, self.model.state_dict(), num_epochs=local_epochs, learning_rate=learning_rate)
                loss_client, acc_client = client.client_evaluate(val=True)
                
                # store it for SecAgg
                self.updated_params_cache.append(updated_params_list)
                self.weights_cache.append(weight)
                self.model_dict.append(model_dict_list)
                
                #store it for FairFed agg
                self.acc_clients.append(acc_client)

            
            # self.aggregate_parameters_FedAvg(self.updated_params_cache, self.weights_cache)
            self.aggregate_parameters_FedAvg_updated(self.model_dict)
            self.aggregate_parameters_FedFair(self.updated_params_cache, self.weights_cache,self.acc_clients)
                
            torch.save(self.global_params_dict, os.path.join(self.temp_dir, "global_model.pt"),)

        # Test Clients Models and Global Model
        loss_client, acc_client = client.client_evaluate()
        loss_server, acc_server = client.server_evaluate()
    

    @torch.no_grad()
    def aggregate_parameters_FedAvg(self, updated_params_cache, weights_cache):
        print("weights_cache::",weights_cache)
        weight_sum = sum(weights_cache)  
        
        weights = torch.tensor(weights_cache, device=self.device) / weight_sum
        
        aggregated_params = []

        for params in zip(*updated_params_cache):
            aggregated_params.append(
                torch.sum(weights * torch.stack(params, dim=-1), dim=-1)
            )

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), aggregated_params)
        )

    
    @torch.no_grad()
    def aggregate_parameters_FedAvg_updated(self, model_dict_list):        
        w_avg = copy.deepcopy(model_dict_list[0])
        for k in w_avg.keys():
            for i in range(1, len(model_dict_list)):
                w_avg[k] += model_dict_list[i][k]
            w_avg[k] = torch.div(w_avg[k], len(model_dict_list))

        self.global_params_dict = OrderedDict(
            zip(self.global_params_dict.keys(), w_avg)
        )
        
        self.global_model.load_state_dict(w_avg)
        torch.save(self.global_model.state_dict(), os.path.join(self.temp_dir, "global_model_2.pt"),)
            

    # def global_acc(self,acc_clients):
    #     #global model has to be there 
        
    #     sum_acc_client=sum(acc_clients)

        
    # # def calc_fairness(self,):
        
    # @torch.no_grad()
    def aggregate_parameters_FedFair (self, updated_params_cache,weights_cache,acc_clients):
        print("aggregate_parameters_FedFair")
        # if not fair_local_client:
        #     global_acc(acc_clients)
        # else:
        #     Delta_t_C_i = abs(fair_global_t_step - fair_local_client) 
        
model = DeepNet()
server = Serverbase(model)
server.fairFed(num_rounds=2,local_epochs=2)    

  0%|                                                                                                                       | 0/2 [00:00<?, ?it/s]

cpu
self.client_id::  0
epoch 0
Epoch 1/2, Training Accuracy: 0.72%
epoch 1
Epoch 2/2, Training Accuracy: 0.74%
------validation starts--------
 selected client for Testing::  0
 Accuracy: 75.56%
Precision: 63.39%
Recall: 87.73%



 selected client for Testing::  1
 Accuracy: 75.87%
Precision: 65.92%
Recall: 78.40%



cpu
self.client_id::  1
epoch 0
Epoch 1/2, Training Accuracy: 0.61%
epoch 1


 50%|███████████████████████████████████████████████████████▌                                                       | 1/2 [00:02<00:02,  2.60s/it]

Epoch 2/2, Training Accuracy: 0.67%
------validation starts--------
 selected client for Testing::  0
 Accuracy: 73.74%
Precision: 61.26%
Recall: 87.57%



 selected client for Testing::  1
 Accuracy: 69.80%
Precision: 62.66%
Recall: 54.52%



aggregate_parameters_FedFair
cpu
self.client_id::  0
epoch 0
Epoch 1/2, Training Accuracy: 0.72%
epoch 1
Epoch 2/2, Training Accuracy: 0.75%
------validation starts--------
 selected client for Testing::  0
 Accuracy: 76.13%
Precision: 68.50%
Recall: 71.33%



 selected client for Testing::  1
 Accuracy: 71.15%
Precision: 69.22%
Recall: 46.33%



cpu
self.client_id::  1
epoch 0
Epoch 1/2, Training Accuracy: 0.61%
epoch 1


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.59s/it]

Epoch 2/2, Training Accuracy: 0.68%
------validation starts--------
 selected client for Testing::  0
 Accuracy: 77.68%
Precision: 71.19%
Recall: 71.19%



 selected client for Testing::  1
 Accuracy: 73.52%
Precision: 62.17%
Recall: 80.79%



aggregate_parameters_FedFair
--------Testing starts-------
 selected client for Testing::  0
 Accuracy: 76.28%
Precision: 70.88%
Recall: 71.03%



 selected client for Testing::  1
 Accuracy: 73.61%
Precision: 64.08%
Recall: 80.31%



Global Model testing Starts
kindly check the Path. Select it based on FedAvg Model
Global Testing Accuracy: 58.07%
Global Precision: 49.31%
Global Recall: 99.57%


In [9]:
# loss_after, acc_after = server.server_evaluate()

In [10]:
# def FedAvg(w, clients):
        #lenghth of data is clients
#     print(clients)
#     w_avg = copy.deepcopy(w[0])
#     for k in w_avg.keys():
#         for i in range(1, len(w)):
#             tens = torch.mul(w[i][k], clients[i])
#             w_avg[k] += tens
#         w_avg[k] = torch.div(w_avg[k], sum(clients))
#     return w_avg

In [11]:
# def FedAvg(w):
#     w_avg = copy.deepcopy(w[0])
#     for k in w_avg.keys():
#         for i in range(1, len(w)):
#             w_avg[k] += w[i][k]
#         w_avg[k] = torch.div(w_avg[k], len(w))
#     return w_avg